# Embedding API

## 1) Data Preprocessing

In [3]:
from langchain_community import embeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts  import ChatPromptTemplate

In [4]:
model_local = ChatOllama(model='phi4')

In [5]:
import json
from pprint import pprint

In [6]:
file1 = "./data/20250201/parsed_rules_20250201.json"

with open(file1, "r", encoding="utf-8") as file:
    data = json.load(file)


In [16]:
documents = []

for ruleType in data.keys():
    rules = data[ruleType]
    for rule in rules:
        page_content = rule
        page_content["type"] = ruleType
    
        newDocument = {
            "page_content": json.dumps(page_content, ensure_ascii=False)}
        documents.append(newDocument)
    

In [17]:
pprint(documents[0])

{'page_content': '{"title": "< 전 문 >", "content": "더불어민주당은 대한민국임시정부의 자주독립정신과 '
                 '4·19혁명, 부마민주항쟁, 5·18민주화운동, 6·10민주항쟁, 촛불시민혁명의 민주주의 정신을 계승하고 '
                 '서민과 중산층을 대변한다.\\n우리는 모든 사람이 공정하고 동등한 조건에서 자신의 역량을 발휘하는 정의로운 '
                 '나라를 원한다. 사회경제적 양극화와 불평등을 극복하고 모든 사람의 기본적인 삶을 보장하는 기본사회를 원한다. '
                 '계층, 세대, 성별, 지역 간 갈등을 해소하고 모든 국민이 조화롭게 살아가는 통합의 국가를 '
                 '원한다.\\n대한민국은 대전환의 시대에 직면해 있다. 기후변화, 세계경제 침체, 국제적 대립 심화, '
                 '인공지능·디지털 전환 가속화는 우리나라에 위기이자 도전으로 작용하고 있다. 이에 혁신성장을 중심으로 국가 '
                 '재도약의 발판을 마련하고, 모든 사람의 평화롭고 더 나은 삶을 보장하는 것이 시대적 과제이다.\\n우리는 '
                 '‘모두 함께 행복한 나라’를 꿈꾼다. 이를 실현하기 위해 ‘공정, 생명, 포용, 번영, 평화’의 5대 가치를 '
                 '추구한다. 우리는 모든 사람이 차별 없이 평등한 기회를 갖는 공정한 사회, 모든 생명의 가치를 중시하는 '
                 '안전한 사회, 사회적 약자를 존중하고 모든 사람의 보편복지를 추구하는 포용의 사회, 모든 사람이 더 풍요로운 '
                 '삶을 누리는 번영의 나라, 한반도를 넘어 글로벌 평화를 선도하는 국가를 만들 것이다.\\n우리는 우리의 꿈과 '
                 '가치를 위해 13개 정책목표를 수립하고 이행한다.\\n하나, ‘혁신

In [18]:
len(documents)

65

## 2) Data Insert

In [19]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

index_name = "test2"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

/Users/ansangjun/Desktop/Projects/python/FastRAG/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [21]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings.OllamaEmbeddings(model="bge-m3"))

/var/folders/yh/12scbgj57_d8xq3nw0klxfq00000gn/T/ipykernel_15272/3115346873.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vector_store = PineconeVectorStore(index=index, embedding=embeddings.OllamaEmbeddings(model="bge-m3"))


In [23]:
from langchain_core.documents import Document

vectorDocuments = []
for document in documents:
    vectorDocument = Document(page_content=document["page_content"])
    vectorDocuments.append(vectorDocument)

In [24]:
ids = [str(i) for i in range(498, len(vectorDocuments)+499)]
vector_store.add_documents(documents=vectorDocuments, ids=ids)

API call failed after 3 attempts: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 04 Feb 2025 17:27:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '787', 'x-pinecone-request-id': '6419566380661246544', 'x-envoy-upstream-service-time': '5', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 56084 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}



PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 04 Feb 2025 17:27:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '787', 'x-pinecone-request-id': '6419566380661246544', 'x-envoy-upstream-service-time': '5', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 56084 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}


## 3) Chat Completion Test

In [22]:
results = vector_store.similarity_search(query="황정아",k=3)
print(len(results))
for doc in results:
    pprint(f"* {doc.page_content} [{doc.metadata}]")


3
('* {"소속": "중앙당", "이름": "황정아", "직책": "대변인", "슬로건": ""} [{\'국회\': '
 "'https://www.assembly.go.kr/members/22nd/HWANGJUNGA', '네이버블로그': "
 "'https://blog.naver.com/k0heaven', '유튜브': "
 "'https://www.youtube.com/@hwangjunga2024', '이미지': "
 "'https://theminjoo.kr/people/connect/people/1446/profile.jpg', '인스타그램': "
 "'https://www.instagram.com/junga1004/'}]")
('* {"소속": "국회의원", "이름": "황정아", "직책": "국회의원(대전, 유성구을)", "슬로건": ""} [{\'국회\': '
 "'https://www.assembly.go.kr/members/22nd/HWANGJUNGA', '네이버블로그': "
 "'https://blog.naver.com/k0heaven', '유튜브': "
 "'https://www.youtube.com/@hwangjunga2024', '이미지': "
 "'https://theminjoo.kr/people/connect/people/1446/profile.jpg', '인스타그램': "
 "'https://www.instagram.com/junga1004/'}]")
('* {"type": "서면브리핑", "title": "[황정아 대변인] 여론조사검증제도개선특위 발족, 건강한 공론장을 위한 과제들을 '
 '추진해 나가겠습니다", "content": "황정아 대변인 서면브리핑\\n■\\n여론조사검증제도개선특위 발족, 건강한 공론장을 위한 '
 '과제들을 추진해 나가겠습니다\\n\\n여론조사는 국민의 생각을 담아내는 중요한 공적 자원입니다. 정책과 정치의 방향을 결정하는 데 '
 '활용되며, 유권자가 올바른 판단을 내릴 수 있도록 돕는 민주사회의 필수

In [23]:
retriever = vector_store.as_retriever()

after_rag_template = """Refer to the content below and respond in Korean. Ensure that your response sounds natural, as if you are directly providing the answer yourself.:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)

In [24]:
print(after_rag_chain.invoke("황정아가 최근에 얘기한거?"))


황정아 대변인은 최근 두 가지 중요한 이슈를 언급했습니다. 첫 번째는 여론조사검증제도의 개선입니다. 그녀는 최근 여론조사가 질문 문항, 응답률 및 편향성 측면에서 여러 논란이 제기되고 있다고 지적하며, 이를 해결하기 위해 여론조사검증제도를 개선할 것을 발표했습니다. 이는 국민의 신뢰를 회복시키고 정확한 공공 의견을 반영하는 데 중요하다고 강조했습니다.

두 번째로 언급된 것은 설 명절 기간 동안 발생한 내란 문제와 그에 따른 경제 및 민생 회복입니다. 황정아는 이러한 상황이 경제와 국민의 생활에 큰 부담을 주고 있다며, 이를 조기 종식하고 민생 경제를 살리겠다는 명확한 입장을 밝혔습니다. 그녀는 내란 세력들이 헌법과 법치에 대해 어긋나는 행동을 하고 있다며 비판하면서, 국민의 요구를 충족시키기 위한 책임 있는 조치가 필요하다고 강조했습니다.

이 두 가지 주제 모두 황정아 대변인은 정부와 시민사회에게 즉각적인 관심과 대응을 요구하며, 국가의 안정성과 국민의 복지를 최우선으로 여기고 있다는 입장을 분명히 했습니다.
